In [1]:
import json
import re

In [2]:
split = 'train'

In [3]:
with open(f'svamp_{split}_llama3.json') as f:
    llama_output = json.load(f)

In [4]:
with open(f'svamp_{split}_matches.txt', 'r') as f:
    matches = f.readlines()

In [ ]:
with open(f'svamp_{split}_random_llama3.json') as f:
    llama_output = json.load(f)

with open(f'svamp_{split}_matches.txt', 'r') as f:
    matches = f.readlines()

In [5]:
def curate(data, index):
    # Extracting fields
    gen_label = data[index]['gen_label'].lower().replace('rationale:', '').strip()
    org_label = data[index]['org_label']
    # choices = data[index]['input'].split('\n')[-5:]
    # split_choice = [re.split(r'\(.\)', choice)[1].strip() for choice in choices]
    # derived_label = "".join(choices[j] for j in range(len(split_choice)) if split_choice[j] in gen_label)

    # Splitting rationale and label
    if ('the answer is' in gen_label and len(gen_label) > 1):
        try:
            rationale, label = gen_label.split('the answer is')
        except:
            rationale = gen_label.split('the answer is')[0]
            label = ""
    else:
        rationale = gen_label
        label = ""

    # Preparing the rationale
    if rationale == "":
        rationale = gen_label.replace("the answer is","").replace('answer:', '').strip()    
    rationale = rationale.replace('hence', '').strip().strip('.')

    # Preparing the label
    if org_label not in label:
        label = org_label
    
    return rationale, label

In [6]:
file_num = 0
dumper = []

for i in range(len(llama_output)):
    
    rationale, label = curate(llama_output, i)

    matched_num = int(matches[i].split()[0])
    rag_rationale, rag_label = curate(llama_output, matched_num)
    rag_question = llama_output[matched_num]['input']
    rag_answer = f"{rag_rationale}. So the answer is {rag_label}"
    
    entry = f"{rationale}. So the answer is {label}. Q: {rag_question} A: {rag_answer}"
    dumper.append(entry)

    if i % 500 == 499 or i == (len(llama_output) - 1):
        with open(f"./llm/{split}_CoT_{file_num}.json", "w") as f:
            json.dump(dumper, f)
        dumper = []
        file_num += 1

to find the number of points luke scored per round, we divide the total points scored by the number of rounds played. so, 84 points / 2 rounds = 42 points per round. answer: 42. So the answer is ( 84.0 / 2.0 ). Q: Luke played a trivia game and gained 3 points in each round of a game. If he scored 78 points in the trivia game
How many rounds did he play? A: to find the number of rounds luke played, we use the formula total points scored = number of rounds played * points gained per round, so the number of rounds played is total points scored divided by points gained per round. in this case, 78 / 3 = 78 rounds. So the answer is ( 78.0 / 3.0 )
30 crayons per row * 7 rows = 210 crayons. So the answer is ( 7.0 * 30.0 ). Q: Faye was placing her pencils and crayons into 11 rows with 31 pencils and 27 crayons in each row.
How many pencils and crayons does she have altogether? A: 31 pencils * 11 rows + 27 crayons * 11 rows = 362 pencils and 297 crayons altogether. answer: 362 pencils and 297 cr